In [2]:
import os
import math
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import f1_score

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_path="data/dataset_train_2024.csv"):
        
        data = pd.read_csv(csv_path)
        
        self.sequences_1 = data.iloc[:, 1:129].values * 100  # Columns 1-128 (1-based indexing)
        self.sequences_2 = data.iloc[:, 129:257].values * 100  # Columns 129-256
        self.extra_feature = data.iloc[:, 257].values.reshape(-1, 1)  # Column 257

        all_features = np.hstack([self.sequences_1, self.sequences_2, self.extra_feature])
        
        self.scaler = StandardScaler()
        self.normalized_features = self.scaler.fit_transform(all_features)
        self.features = torch.tensor(self.normalized_features, dtype=torch.float32)

        self.label_encoder = LabelEncoder()
        self.labels = torch.tensor(self.label_encoder.fit_transform(data.iloc[:, -1]), dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
    def inverseTransform(self, array):
        return self.label_encoder.inverse_transform(array)

In [4]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional=False, dropout=0.0):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout
        )

        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [5]:
def reformat_tensor(tensor):
    batch_size = tensor.shape[0]
    seq1 = tensor[:, :128]
    seq2 = tensor[:, 128:256]
    noise = tensor[:, -1].unsqueeze(1).expand(batch_size, 128)
    return torch.stack([seq1, seq2, noise], dim=2)

In [ ]:
hidden_dim = 256
num_layers = 3
bidirectional = True
num_classes = 5
learning_rate = 0.001
epochs = 36
batch_size = 32

dataset = CustomDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = RNNClassifier(
    input_dim=3,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional,
    dropout=0.3
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features = reformat_tensor(features).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Model validation

In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        features = reformat_tensor(features).to(device)
        labels = labels.to(device)

        outputs = model(features)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Prepare for Kaggle


In [ ]:
class UnlabeledDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe

        self.scaler = StandardScaler()
        normalized_values = self.scaler.fit_transform(self.data.values)
        self.normalized_data = pd.DataFrame(
            normalized_values, columns=self.data.columns, index=self.data.index
        )
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.normalized_data.iloc[idx].values.astype('float32')  # Retrieve row as NumPy array
        if self.transform:
            inputs = self.transform(inputs)
        return inputs

csv_path = "data/dataset_test_no_label_2024.csv"  # Path to the dataset CSV file
unlabeled_df = pd.read_csv(csv_path)  # Update the filename
unlabeled_df = unlabeled_df.drop(unlabeled_df.columns[0], axis=1)
unlabeled_dataset = UnlabeledDataset(unlabeled_df)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

model.eval()

predictions = []
indices = []

with torch.no_grad():
    for idx, inputs in enumerate(unlabeled_dataloader):
        inputs = inputs.to(device)  # Send inputs to the same device as the model
        
        inputs = reformat_tensor(inputs)
        
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        
        start_idx = idx * unlabeled_dataloader.batch_size
        batch_indices = list(range(start_idx, start_idx + len(inputs)))
        indices.extend(batch_indices)
        predictions.extend(preds.cpu().numpy())

output_df = pd.DataFrame({"ID": indices, "MODULATION": dataset.inverseTransform(predictions)})

output_df.to_csv("./CSV/Spredictions_RNN_4.csv", index=False)

print("Predictions saved to 'predictions_RNN.csv'")

# Save the model

In [ ]:
model_path = "./RNN/rnn_classifier_gridSearch_3_best.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Load saved model

In [ ]:
input_dim = 3
hidden_dim = 128
num_layers = 2
num_classes = 5
bidirectional = False

loaded_model = RNNClassifier(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional
)

loaded_model.load_state_dict(torch.load("rnn_classifier.pth"))
loaded_model.to(device)
loaded_model.eval()

print("Model loaded successfully!")

# GridSearch


In [22]:
def train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            features = reformat_tensor(features)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            features = reformat_tensor(features)
            outputs = model(features)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    score = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, score

In [ ]:
from itertools import product

hidden_dims = [32, 64, 128, 256]
num_layers = [2, 3, 4, 5, 6]
learning_rates = [0.001]
batch_sizes = [32]
dropouts = [0.2, 0.5]
bidirectionals = [True]
num_classes = 5
epochs = 35
    
best_accuracy = 0
best_f1_score = 0
best_params = None

dataset = CustomDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

hyperparameter_combinations = product(hidden_dims, num_layers, learning_rates, batch_sizes, dropouts, bidirectionals)

for hidden_dim, num_layer, learning_rate, batch_size, dropout, bidirectional in hyperparameter_combinations:
        print(f"Training with hidden_dim={hidden_dim}, num_layers={num_layer}, lr={learning_rate}, batch_size={batch_size}, dropout={dropout}, bidirectional={bidirectional}")
        
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

        model = RNNClassifier(input_dim=3, hidden_dim=hidden_dim, num_layers=num_layer, num_classes=num_classes, 
                              bidirectional=bidirectional, dropout=dropout).to(device)
        
        accuracy, score = train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size)
        
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation F1-Score: {score:.4f}")
        
        if score > best_f1_score:
            best_accuracy = accuracy
            best_f1_score = score
            best_params = {
                  'hidden_dim': hidden_dim,
                  'num_layers': num_layer,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'dropout': dropout,
                  'bidirectional': bidirectional
                  }

print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

In [ ]:
print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

RNN_GridSearch

Best Parameters found: {'hidden_dim': 32, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 16, 
'dropout': 0.0, 'bidirectional': True}

Best Validation Accuracy: 0.9900

Best Validation F1-Score: 0.9900

RNN_2 - Best one on kaggle

Training with hidden_dim=64, num_layers=4, lr=0.001, batch_size=32, dropout=0.0, bidirectional=True

Validation Accuracy: 0.9925

Validation Accuracy: 0.9925

Training with hidden_dim=256, num_layers=3, lr=0.001, batch_size=32, dropout=0.0, bidirectional=True

Validation Accuracy: 0.9929

Validation Accuracy: 0.9929

GridSearch 3:

Best Parameters found: {'hidden_dim': 256, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 32, 'bidirectional': True, dropout: 0.2}

Best Validation Accuracy: 0.9912

Best Validation F1-Score: 0.9912